In [1]:
import os 
os.chdir(r'Q:\sachuriga\Sachuriga_Python/quattrocolo-nwb4fp\src')

from neurochat.nc_data import NData
from neurochat.nc_spike import NSpike
from neurochat.nc_spatial import NSpatial
import neurochat.nc_plot as nc_plot
from neurochat.nc_lfp import NLfp
import matplotlib.pyplot as plt
import numpy as np
from pynwb import NWBHDF5IO
import matplotlib.pyplot as plt
import numpy as np
import math
import pynapple as nap
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import normalize

import sys
import nwb4fp.analyses.maps as mapp
from nwb4fp.analyses.examples.tracking_plot import plot_ratemap,plot_path
from nwb4fp.analyses.fields import separate_fields_by_laplace, separate_fields_by_dilation,find_peaks,separate_fields_by_laplace_of_gaussian,calculate_field_centers,distance_to_edge_function, remove_fields_by_area, map_pass_to_unit_circle,which_field,compute_crossings
from elephant.statistics import time_histogram, instantaneous_rate
from nwb4fp.analyses import maps
from nwb4fp.analyses.data import pos2speed,speed_filtered_spikes,load_speed_fromNWB,load_units_fromNWB,get_filed_num,unit_location_ch
from scipy.ndimage import gaussian_filter
import ast
import pandas as pd
pd.set_option('display.max_rows', None)
np.set_printoptions(threshold=np.inf)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

Q:\sachuriga\Sachuriga_Python\quattrocolo-nwb4fp\src\nwb4fp\analyses\examples\tracking_plot.py:5: DeprecationWarning: Please import `center_of_mass` from the `scipy.ndimage` namespace; the `scipy.ndimage.measurements` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.measurements import center_of_mass


In [2]:
df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements_with_stability.pkl')
df_good = df_loaded[df_loaded['unit_quality'] == "good"]
df_a  = df_good[df_good['session'] == "A"]

metrics = ['l_ratio', 'isi_violations_ratio', 
        'amplitude_median', 'snr']

titles = ['L ratio', 'ISI violations ratio', 
        'Amplitude median (mV)', 'Signal to noise ratio']

fig = plt.figure(figsize=(7.2, 2.4), dpi=2400)
plt.rcParams.update({'font.size': 8,'font.family': 'DejaVu Sans'})
gs = gridspec.GridSpec(1, 4, height_ratios=[.8], width_ratios=[0.8, .8, .8, .8])  # First row taller
plt.rcParams.update({
    'axes.labelpad': -0.1,
    'ytick.major.pad': -0.1,
    'xtick.major.pad': -0.1,
    'ytick.major.size': 2,
    'xtick.major.size': 2
})

ax1_1 = fig.add_subplot(gs[0, 0])
ax1_2 = fig.add_subplot(gs[0, 1])
ax1_3 = fig.add_subplot(gs[0, 2])
ax1_4 = fig.add_subplot(gs[0, 3])

# ax2_1 = fig.add_subplot(gs[1, 0])
# ax2_2 = fig.add_subplot(gs[1, 1])
# ax2_3 = fig.add_subplot(gs[1, 2])
# ax2_4 = fig.add_subplot(gs[1, 3])
test_hy = ['two-sided','two-sided','two-sided','two-sided']
idx=0

for ax in [ax1_1 ,ax1_2 ,ax1_3 ,ax1_4]:
    # Load Good units

    base_folder = r"Q:/sachuriga/CR_CA1_paper/Results/functional_cell_type"
    control_ids = ['65165', '65091', '63383', '66539', '65622']
    exp_ids = ['65588', '63385', '66538', '66537', '66922']

    # Separate into control and experimental groups
    control_df = df_a[df_a['animal_id'].isin(control_ids)]
    exp_df = df_a[df_a['animal_id'].isin(exp_ids)]

    # Statistical comparisons for scalar metrics

    # Define custom colors
    control_color = 'blue'  # Dark blue for Control
    exp_color = "red"  # Light blue for Experimental
    metric=metrics[idx]
    control_values = control_df[metric].dropna()
    exp_values = exp_df[metric].dropna()
    
    if len(control_values) > 0 and len(exp_values) > 0:
        control_mean = control_values.mean()
        exp_mean = exp_values.mean()
        control_sem = control_values.sem()
        exp_sem = exp_values.sem()
        print(f"\nComparison for {metric}:")
        print(f"Control mean: {control_mean:.2f} ± {control_sem:.2f}")
        print(f"Experimental mean: {exp_mean:.2f} ± {exp_sem:.2f}")
        
        # Mann-Whitney U test
        control_array = np.asarray(control_values.values, dtype=float)
        exp_array = np.asarray(exp_values.values, dtype=float)

        # Remove NaN values
        control_clean = control_array[~np.isnan(control_array)]
        exp_clean = exp_array[~np.isnan(exp_array)]

        u_stat, p_val = stats.mannwhitneyu(control_clean, exp_clean, alternative = test_hy[idx])

        # Prepare data for Seaborn plotting
        plot_df = pd.DataFrame({
            'value': pd.concat([control_values, exp_values]),
            'group': ['Control'] * len(control_values) + ['Experimental'] * len(exp_values)
        })
        
        # Check if deviation is "too large" (using coefficient of variation > 1 as threshold)
        all_values = plot_df['value']
        cv = all_values.std() / all_values.mean()  # Coefficient of variation
        use_log_scale = cv > 1 and all_values.min() > 0  # Ensure positive values for log scale
        
        # Filter out outliers (e.g., beyond 3 standard deviations)
        mean_val = all_values.mean()
        std_val = all_values.std()
        plot_df_filtered = plot_df[(plot_df['value'] >= mean_val - 3 * std_val) & 
                                (plot_df['value'] <= mean_val + 3 * std_val)]
        
        sns.violinplot(
                data=plot_df_filtered, x='group', y='value', ax=ax,inner = None,
                palette={"Control": control_color, "Experimental": exp_color}, width=0.8, cut=0, edgecolor='white'
            )
        sns.boxplot(
                data=plot_df_filtered, 
                x='group', y='value',
                palette={"Control": "black", "Experimental": "black"},
                width=0.3, 
                fill=False,  # No fill, only outlines
                showfliers=False,  # Hide outliers
                showmeans=False,  # Remove mean marker, assuming midline is the median
                linewidth=1,  # Makes the lines narrower (thinner)
                ax=ax  # Add this
            )
        # Add individual points with matching colors
        # sns.swarmplot(
        #             data=plot_df_filtered, x='group', y='value', ax=ax, size=0.5,
        #             hue='group', palette={"Control": "black", "Experimental": "black"},
        #             alpha=1, legend=False
        #         )
        ax.set_ylabel(titles[idx])
        ax.set_xlabel('')
        ax.yaxis.grid(False)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(True)
        ax.spines['left'].set_visible(True)
        ax.set_xticklabels(['CR;DTA-', 'CR;DTA+'], rotation = -30)
        y_max = ax.get_ylim()[1]
        bar_height = y_max * 0.1  # Adjust this value to position the bar above the plot
        x_positions = [0, 1]  # Adjusted positions for 'Control' and 'Experimental' groups
        if (p_val < 0.05) & (p_val > 0.01):
            ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                                color='black', lw=1.5)
            ax.text(0.5, y_max + bar_height * 1.1, f'*', ha='center', va='bottom')
        elif (p_val < 0.01) & (p_val > 0.001):
            ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                                color='black', lw=1.5)
            ax.text(0.5, y_max + bar_height * 1.1, f'**', ha='center', va='bottom')
        elif p_val < 0.001:
            ax.plot([x_positions[0], x_positions[1]], [y_max + bar_height, y_max + bar_height], 
                                color='black', lw=1.5)
            ax.text(0.5, y_max + bar_height * 1.1, f'***', ha='center', va='bottom')
    idx += 1
    print(idx)


fig.subplots_adjust(top=0.92, bottom=0.08, left=0.1, right=0.95, hspace=1.1, wspace=1.3)
plt.tight_layout()
plt.savefig(r'Q:\sachuriga\CR_CA1_paper\Figures/suppfig2.png', transparent=True, dpi=1200, bbox_inches='tight')
# def remove_outliers(df, column):
#     """Remove outliers using IQR method"""
#     Q1 = df[column].quantile(0.25)
#     Q3 = df[column].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]



# df_loaded = pd.read_pickle(r'Q:/sachuriga/CR_CA1_paper/tables/functional_properties_with_python_measurements.pkl')
# # Separate into control and experimental groups

# df_good = df_loaded[df_loaded['unit_quality']=="good"]
# df_a = df_good[(df_good['buzaki_py_cell_type']=="pyramidal")&(df_good['session']=="A")]
# control_df = df_a[df_a['animal_id'].isin(control_ids)]
# exp_df = df_a[df_a['animal_id'].isin(exp_ids)]

# # Metrics for analysis
# metrics = ['l_ratio', 'isi_violations_ratio', 
#         'amplitude_median', 'snr']
# metrics1  = ['mean_firing_rate']

# # metrics = ['amplitude_median','half_width','peak_to_valley','peak_trough_ratio', 'recovery_slope', 'repolarization_slope', 
# #             'mean_firing_rate',  'mean_inter_spike_interval','bursting_index' ]

# axes = [ax2_1 ,ax2_2 ,ax2_3 ,ax2_4]

# # Define custom colors
# control_color = "blue"
# exp_color = "red"

# for idx, metric in enumerate(metrics):
#     # Prepare and clean control group data
#     control_data = pd.concat([control_df[metrics1[0]], control_df[metric]], axis=1).dropna()
#     # control_data = remove_outliers(control_data, metrics1[0])
#     # control_data = remove_outliers(control_data, metric)
    
#     # Prepare and clean experimental group data
#     exp_data = pd.concat([exp_df[metrics1[0]], exp_df[metric]], axis=1).dropna()
#     # exp_data = remove_outliers(exp_data, metrics1[0])
#     # exp_data = remove_outliers(exp_data, metric)

#     if len(control_data) > 2 and len(exp_data) > 2:  # Need at least 3 points for meaningful fit
#         # Linear regression for control
#         # Assuming control_data is a pandas DataFrame or similar
#         x = control_data[metrics1[0]]
#         y = control_data[metric]

#         # Convert to NumPy arrays and ensure they are numeric
#         x = np.array(x, dtype=float)
#         y = np.array(y, dtype=float)

#         # Check for valid lengths
#         if len(x) != len(y):
#             raise ValueError(f"Mismatched lengths: len(x) = {len(x)}, len(y) = {len(y)}")

#         # Check for NaN or infinite values
#         if np.any(np.isnan(x)) or np.any(np.isnan(y)):
#             raise ValueError("Input arrays contain NaN values")
#         if np.any(np.isinf(x)) or np.any(np.isinf(y)):
#             raise ValueError("Input arrays contain infinite values")

#         # Check if arrays have enough data points
#         if len(x) < 2:
#             raise ValueError("Input arrays must have at least 2 data points")

#         # Perform linear regression
#         control_slope, control_intercept, control_r, control_p, control_se = stats.linregress(x, y)

#         # control_slope, control_intercept, control_r, control_p, control_se = stats.linregress(
#         #     control_data[metrics1[0]], control_data[metric])

#         x1 = exp_data[metrics1[0]]
#         y1 = exp_data[metric]
#         x1 = np.array(x1, dtype=float)
#         y1 = np.array(y1, dtype=float)
#         exp_slope, exp_intercept, exp_r, exp_p, exp_se = stats.linregress(
#             x1, y1)

#         # Print results
#         print(f"Control: slope = {control_slope:.4f} ± {control_se:.4f}, R² = {control_r**2:.4f}, n = {len(control_data)}")
#         print(f"Experimental: slope = {exp_slope:.4f} ± {exp_se:.4f}, R² = {exp_r**2:.4f}, n = {len(exp_data)}")


#                 # Given
#         slope_diff = control_slope - exp_slope
#         se_diff = np.sqrt(control_se**2 + exp_se**2)
#         z_stat = slope_diff / se_diff

#         # Calculate the two-tailed p-value
#         p_value = 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
#         print(f"Slope difference t-statistic: {p_value :.4f}, p-value: {p_value:.4f}")

#         # Create scatter plot with regression lines
#         # Control group
#         control_plot = sns.scatterplot(
#             data=control_data, 
#             x=metrics1[0], 
#             y=metric, 
#             ax=axes[idx], 
#             color=control_color,
#             label='CRs +',
#             s=2
#         )
#         control_line = axes[idx].plot(control_data[metrics1[0]], 
#                                     control_slope * control_data[metrics1[0]] + control_intercept, 
#                                     color=control_color,
#                                     label=f'CRs + (R² = {control_r**2:.2f})')

#         # Experimental group
#         exp_plot = sns.scatterplot(
#             data=exp_data, 
#             x=metrics1[0], 
#             y=metric, 
#             ax=axes[idx], 
#             color=exp_color,
#             label='CRs -',
#             s=2
#         )
#         exp_line = axes[idx].plot(exp_data[metrics1[0]], 
#                                 exp_slope * exp_data[metrics1[0]] + exp_intercept, 
#                                 color=exp_color,
#                                 label=f'CRs - (R² = {exp_r**2:.2f})')

#         # Customize plot
#         #axes[idx].set_title(f'{metric} vs {metrics1[0]}')
#         axes[idx].set_xlabel(metrics1[0])
#         axes[idx].set_ylabel(metric)
        
#         # Update legend with R² values
#         axes[idx].legend(handles=[control_line[0], exp_line[0]],
#                     labels=[f'CRs + (R² = {control_r**2:.2f})', 
#                             f'CRs - (R² = {exp_r**2:.2f})'])
        

#         # Remove top and right spines
#         axes[idx].spines['top'].set_visible(False)
#         axes[idx].spines['right'].set_visible(False)



Comparison for l_ratio:
Control mean: 0.01 ± 0.00
Experimental mean: 0.01 ± 0.00
1

Comparison for isi_violations_ratio:
Control mean: 0.03 ± 0.00
Experimental mean: 0.02 ± 0.00
2

Comparison for amplitude_median:
Control mean: -441.00 ± 12.76
Experimental mean: -438.57 ± 11.70


C:\Users\sachur\AppData\Local\Temp\ipykernel_27184\3819482083.py:90: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.violinplot(
C:\Users\sachur\AppData\Local\Temp\ipykernel_27184\3819482083.py:94: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(
C:\Users\sachur\AppData\Local\Temp\ipykernel_27184\3819482083.py:118: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(['CR;DTA-', 'CR;DTA+'], rotation = -30)
C:\Users\sachur\AppData\Local\Temp\ipykernel_27184\3819482083.py:90: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hu

3

Comparison for snr:
Control mean: 14.07 ± 0.47
Experimental mean: 14.07 ± 0.42
4


In [ ]:
control_slope, control_intercept, control_r, control_p, control_se = stats.linregress(
    control_data[metrics1[0]], control_data[metric])

In [ ]:
control_data[metric]

In [ ]:
control_data

In [ ]:
print("control_data columns:", control_data.columns)
print("metrics1[0]:", metrics1[0], "metric:", metric)

In [ ]:
control_data[metrics1[0]].values